In [14]:
import sys

sys.path.append('../../src')
from pjplan import Task, WBS, DefaultScheduler, Resource

In [15]:
# Sample project
with WBS() as prj:
    prj // Task(1, 'Task 1', estimate=40, resource='Tester')
    prj // Task(2, 'Task 2', predecessors=[prj[1]], estimate=20, resource='Tester')
    with prj // Task(3, 'Task 3') as t:
        t // Task(4, 'Task 4', predecessors=[prj[2]], estimate=100)
        t // Task(5, 'Task 5', predecessors=[prj[2]], estimate=50)
    with prj // Task(6, 'Task 6') as t:
        t // Task(7, 'Task 7', predecessors=[prj[5]], estimate=16)
        t // Task(8, 'Task 8', predecessors=[prj[7]], estimate=16)

### Simple print

In [16]:
# Print full WBS
prj # or prj.print()

 ID  NAME       RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 1   Task 1     Tester    40        -      -      -    []           
 2   Task 2     Tester    20        -      -      -    [1]          
 3   Task 3     -         -         -      -      -    []           
 4      Task 4  -         100       -      -      -    [2]          
 5      Task 5  -         50        -      -      -    [2]          
 6   Task 6     -         -         -      -      -    []           
 7      Task 7  -         16        -      -      -    [5]          
 8      Task 8  -         16        -      -      -    [7]          

In [17]:
# Print single task
prj[3]

 ID  NAME       RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 3   Task 3     -         -         -      -      -    []           
 4      Task 4  -         100       -      -      -    [2]          
 5      Task 5  -         50        -      -      -    [2]          

In [18]:
# Print children
prj[3].children

 ID  NAME    RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 4   Task 4  -         100       -      -      -    [2]          
 5   Task 5  -         50        -      -      -    [2]          

In [19]:
# Print selected tasks
prj.tasks(resource='Tester')

 ID  NAME    RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 1   Task 1  Tester    40        -      -      -    []           
 2   Task 2  Tester    20        -      -      -    [1]          

In [20]:
# Print predecessors/successors
prj[4].predecessors

 ID  NAME    RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 2   Task 2  Tester    20        -      -      -    [1]          

### Print selected fields

In [21]:
prj.print(['id', 'name', 'resource'])

 ID  NAME       RESOURCE 
 1   Task 1     Tester   
 2   Task 2     Tester   
 3   Task 3     -        
 4      Task 4  -        
 5      Task 5  -        
 6   Task 6     -        
 7      Task 7  -        
 8      Task 8  -        


### Hide children

In [22]:
prj.print(children=False)

 ID  NAME    RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 1   Task 1  Tester    40        -      -      -    []           
 2   Task 2  Tester    20        -      -      -    [1]          
 3   Task 3  -         -         -      -      -    []           
 6   Task 6  -         -         -      -      -    []           


### Color themes

In [25]:
# See https://en.wikipedia.org/wiki/ANSI_escape_code for coloring details
theme = {
    'header_color': '94m', #blue
    'level_colors': ['91m', '92m', '93m']
}
prj.print(theme=theme)

 ID  NAME       RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
 1   Task 1     Tester    40        -      -      -    []           
 2   Task 2     Tester    20        -      -      -    [1]          
 3   Task 3     -         -         -      -      -    []           
 4      Task 4  -         100       -      -      -    [2]          
 5      Task 5  -         50        -      -      -    [2]          
 6   Task 6     -         -         -      -      -    []           
 7      Task 7  -         16        -      -      -    [5]          
 8      Task 8  -         16        -      -      -    [7]          


### Custom task colors

In [26]:
# See https://en.wikipedia.org/wiki/ANSI_escape_code for coloring details
prj.tasks(resource='Tester').print_color='30:41m'
prj

 ID  NAME       RESOURCE  ESTIMATE  SPENT  START  END  PREDECESSORS 
[30:41m 1  [30:41m Task 1    [30:41m Tester   [30:41m 40       [30:41m -     [30:41m -     [30:41m -   [30:41m []           
[30:41m 2  [30:41m Task 2    [30:41m Tester   [30:41m 20       [30:41m -     [30:41m -     [30:41m -   [30:41m [1]          
 3   Task 3     -         -         -      -      -    []           
 4      Task 4  -         100       -      -      -    [2]          
 5      Task 5  -         50        -      -      -    [2]          
 6   Task 6     -         -         -      -      -    []           
 7      Task 7  -         16        -      -      -    [5]          
 8      Task 8  -         16        -      -      -    [7]          

# Visualize resource usage

In [27]:
# Let's create schedule
p = prj.clone()
p.tasks(resource=None).resource='Developer'
p.tasks.print_color = None

plan, usage = DefaultScheduler(resources=[Resource(name='Tester'), Resource(name='Developer')]).calc(p)
plan

 ID  NAME       RESOURCE   ESTIMATE  SPENT  START             END               PREDECESSORS 
 1   Task 1     Tester     40        0      06.04.2023 00:00  13.04.2023 00:00  []           
 2   Task 2     Tester     20        0      13.04.2023 00:00  17.04.2023 12:00  [1]          
 3   Task 3     Developer  150       0      17.04.2023 00:00  11.05.2023 18:00  []           
 4      Task 4  Developer  100       0      17.04.2023 00:00  03.05.2023 12:00  [2]          
 5      Task 5  Developer  50        0      03.05.2023 12:00  11.05.2023 18:00  [2]          
 6   Task 6     Developer  32        0      11.05.2023 18:00  17.05.2023 18:00  []           
 7      Task 7  Developer  16        0      11.05.2023 18:00  15.05.2023 18:00  [5]          
 8      Task 8  Developer  16        0      15.05.2023 18:00  17.05.2023 18:00  [7]          

In [28]:
# Visualize usage table
usage

| DATE     | TESTER | DEVELOPER |
| 23-04-06 | 8.0    | 0.0       |
| 23-04-07 | 8.0    | 0.0       |
| 23-04-08 | 0.0    | 0.0       |
| 23-04-09 | 0.0    | 0.0       |
| 23-04-10 | 8.0    | 0.0       |
| 23-04-11 | 8.0    | 0.0       |
| 23-04-12 | 8.0    | 0.0       |
| 23-04-13 | 8.0    | 0.0       |
| 23-04-14 | 8.0    | 0.0       |
| 23-04-15 | 0.0    | 0.0       |
| 23-04-16 | 0.0    | 0.0       |
| 23-04-17 | 4.0    | 8.0       |
| 23-04-18 | 0.0    | 8.0       |
| 23-04-19 | 0.0    | 8.0       |
| 23-04-20 | 0.0    | 8.0       |
| 23-04-21 | 0.0    | 8.0       |
| 23-04-22 | 0.0    | 0.0       |
| 23-04-23 | 0.0    | 0.0       |
| 23-04-24 | 0.0    | 8.0       |
| 23-04-25 | 0.0    | 8.0       |
| 23-04-26 | 0.0    | 8.0       |
| 23-04-27 | 0.0    | 8.0       |
| 23-04-28 | 0.0    | 8.0       |
| 23-04-29 | 0.0    | 0.0       |
| 23-04-30 | 0.0    | 0.0       |
| 23-05-01 | 0.0    | 8.0       |
| 23-05-02 | 0.0    | 8.0       |
| 23-05-03 | 0.0    | 8.0       |
| 23-05-04 | 0